0. This notebook is the continuation to the preprocessing.ipynb notebook.

1. Import all neccessary modules 

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

2. Import X and y

In [2]:
import torchvision
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [3]:
y = pd.read_csv('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/Arousal.csv')

In [4]:
y = y.iloc[:,[1]]

In [5]:
y

,0
0,5.0
1,6.0
2,7.0
3,8.0
4,7.0
...,...
14271,5.0
14272,8.0
14273,2.0
14274,6.0


In [6]:
y_8000_rows = y[0:100]

In [7]:
y_arr = y_8000_rows.to_numpy()

In [8]:
os.chdir('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis')

NEW APPROACH:

Fit CNN on X

1. Read `n-rows`from X.

In [9]:
chunk = 10
i=0
X=[]
import pandas as pd
for df in pd.read_csv('X_matrix.csv',chunksize = chunk):
    X.append(df)
    i=i+1
    print('Processed {i}/10 chunks'.format(i=i))
    if i == 10:
        break



Processed 1/10 chunks
Processed 2/10 chunks
Processed 3/10 chunks
Processed 4/10 chunks
Processed 5/10 chunks
Processed 6/10 chunks
Processed 7/10 chunks
Processed 8/10 chunks
Processed 9/10 chunks
Processed 10/10 chunks


In [10]:
X = pd.concat(X)

In [11]:
del X [X.columns[0]]

In [12]:
X_arr = X.to_numpy()

In [13]:
X_arr = np.reshape(X_arr,(100,224,224))

In [14]:
X_arr.shape

(100, 224, 224)

Adding new dimension for CNN

In [15]:
X_ = X_arr[:,:,np.newaxis]

In [16]:
X_ = np.swapaxes(X_,2,1)

In [17]:
X_.shape

(100, 1, 224, 224)

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X_,y_arr,test_size=0.2,random_state=42)

Converting X to appropriate Datatype...

In [19]:
X_tensor_train = torch.from_numpy(X_train)
X_tensor_test = torch.from_numpy(X_test)
y_tensor_train = torch.from_numpy(y_train)
y_tensor_test = torch.from_numpy(y_test)

In [20]:
X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)
X_tensor_test = torch.tensor(X_tensor_test,dtype = torch.float32)
y_tensor_train = y_tensor_train.type(torch.LongTensor)
y_tensor_test = y_tensor_test.type(torch.LongTensor)

/tmp/ipykernel_179/3712994864.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)
/tmp/ipykernel_179/3712994864.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor_test = torch.tensor(X_tensor_test,dtype = torch.float32)


Pytorch needs the true labels to be in the range N-1. Thus, substract 1 one from each label.

In [21]:
y_tensor_train = y_tensor_train-1
y_tensor_test = y_tensor_test-1

In [22]:
print('training shape X is {}, test shape X is{}'.format(X_tensor_train.shape,X_tensor_test.shape))

training shape X is torch.Size([80, 1, 224, 224]), test shape X istorch.Size([20, 1, 224, 224])


Setup custom dataset class

In [23]:
class CustomDataset(Dataset):
    def __init__(self,X_path, y_path):#, y_path):
        df_x = pd.read_csv(X_path)
        df_y = pd.read_csv(y_path)#'/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/Arousal.csv')
        del X [X.columns[0]]
        X = X.to_numpy()
        X = np.reshape(X,(100,224,224))
        X = X[:,:,np.newaxis]
        X = np.swapaxes(X,2,1)
        y = df_y.iloc[:,[1]]
        
        self.X_train = X
        self.y_train = y.to_numpy()
        
        
    def __len__(self):
        return len(self.y_train)#X)
    
    def __getitem__(self, idx):
        X_train = self.X[idx]
        y_train= self.y_train[idx]
        return X_train, y_train


In [24]:
class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

In [27]:
import torch.nn as nn

In [28]:
dataset_train = SimpleDataset(X_tensor_train,y_tensor_train)
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)

dataset_test = SimpleDataset(X_tensor_test,y_tensor_test)
dataloader_test = DataLoader(dataset_test, batch_size = 32,shuffle=True)
criterion = nn.CrossEntropyLoss()

dataset = CustomDataset('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/X_matrix.csv','/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/Arousal.csv')

dataset = CustomDataset(X_path ='/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/X_matrix.csv', y_path='/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/Arousal.csv')
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

Setting up the CNN.

In [29]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(774400, 128)
        self.fc2 = nn.Linear(128, 10)
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


In [30]:
SalienceNet = CNN()

In [49]:
def analysis(net, loss_fn, dataloader_train,dataloader_test,
          epoch=100, learning_rate=1):
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)#,weight_decay=1e-4)
    train_loss = []
    
    for i in range(epoch):   
        net.train()
        for data, label in dataloader_train:
            data, label = data.to('cpu'), label.to('cpu')
        #make prediction
            output = net(data)

            # Zero out the gradients
            optimizer.zero_grad()

            # Calculate loss.
            loss = criterion(output,torch.max(label, 1)[0])

            # Backpropagation and gradient update.
            loss.backward()# Calculate gradients. #retrain_graph=True

            optimizer.step() # Apply gradient udpate.

            train_loss.append(loss.item())
            
            if (i + 1) % (epoch // 5) == 0:
                print(f'iteration {i + 1}/{epoch} | loss: {loss.item():.3f}')
    
    net.eval()
    correct = 0
    total = 0
    for data, labels in dataloader_test:
        data, labels = data.to('cpu'), labels.to('cpu')
        outputs = net(data)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    return acc


In [42]:
import matplotlib.pyplot as plt

In [50]:
np.random.seed(1)
torch.manual_seed(1)

test_acc=analysis(net=SalienceNet,loss_fn=criterion,dataloader_train=dataloader_train,
                  dataloader_test=dataloader_test,epoch=5)

plt.plot(p)

plt.xlabel('Iterations of gradient descent')
plt.ylabel('Cross entropy')
plt.show()

iteration 1/5 | loss: 1.788
iteration 1/5 | loss: 56.222
iteration 1/5 | loss: 9.945
iteration 2/5 | loss: 1.851
iteration 2/5 | loss: 1.749
iteration 2/5 | loss: 2.179
iteration 3/5 | loss: 2.046
iteration 3/5 | loss: 1.849
iteration 3/5 | loss: 1.712
iteration 4/5 | loss: 1.712
iteration 4/5 | loss: 1.586
iteration 4/5 | loss: 2.070
iteration 5/5 | loss: 1.479
iteration 5/5 | loss: 2.054
iteration 5/5 | loss: 2.078


NameError: name 'p' is not defined